In [ ]:
from src.google_flight_analysis.scrape import Scrape

: 

In [6]:
flights = Scrape("MUC", "LAX", "2023-10-28", export=True)
flights.run_scrape()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: headless chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000105333f48 chromedriver + 4226888
1   chromedriver                        0x000000010532c4f4 chromedriver + 4195572
2   chromedriver                        0x0000000104f70d68 chromedriver + 281960
3   chromedriver                        0x0000000104f59db4 chromedriver + 187828
4   chromedriver                        0x0000000104f7ad54 chromedriver + 322900
5   chromedriver                        0x0000000104fe2e30 chromedriver + 749104
6   chromedriver                        0x0000000104f9ff1c chromedriver + 474908
7   chromedriver                        0x0000000104fa0ef4 chromedriver + 478964
8   chromedriver                        0x00000001052f559c chromedriver + 3970460
9   chromedriver                        0x00000001052f96

In [3]:
flights.data

,departure_datetime,arrival_datetime,airlines,travel_time,origin,destination,layover_n,layover_time,layover_location,price_eur,price_trend,price_value,access_date,one_way,has_train,days_advance
0,2023-10-28 10:25:00,2023-10-29 15:10:00,[Separate tickets booked together],2265,MUC,LAX,1,1450.0,LGW,470,typical,None,2023-07-09 18:00:05.526143,True,False,110
1,2023-10-28 20:37:00,2023-10-28 07:15:00,"[Delta, KLM, Virgin Atlantic]",1087,MUC,LAX,1,195.0,DTW,671,typical,None,2023-07-09 18:00:05.526150,True,False,111
2,2023-10-28 13:10:00,2023-10-28 07:00:00,"[Air France, Delta]",895,MUC,LAX,1,100.0,CDG,676,typical,None,2023-07-09 18:00:05.526153,True,False,110
3,2023-10-28 11:50:00,2023-10-28 07:15:00,[KLMDelta],830,MUC,LAX,1,75.0,AMS,758,typical,None,2023-07-09 18:00:05.526155,True,False,110
4,2023-10-28 18:05:00,2023-10-28 07:00:00,"[Air France, Delta]",1190,MUC,LAX,1,380.0,CDG,674,typical,None,2023-07-09 18:00:05.526157,True,False,111
5,2023-10-28 15:24:00,2023-10-28 11:50:00,"[KLM, Delta]",1044,MUC,LAX,2,NaN,None,675,typical,None,2023-07-09 18:00:05.526159,True,False,110
6,2023-10-28 19:59:00,2023-10-28 22:00:00,"[KLM, Delta]",1029,MUC,LAX,2,3000.0,min,675,typical,None,2023-07-09 18:00:05.526161,True,False,111
7,2023-10-28 22:00:00,2023-10-30 15:25:00,"[Lufthansa, LOT]",3025,MUC,LAX,2,NaN,None,787,typical,None,2023-07-09 18:00:05.526166,True,False,111
8,2023-10-28 21:10:00,2023-10-28 11:50:00,"[Air France, Delta]",885,MUC,LAX,1,80.0,CDG,875,typical,None,2023-07-09 18:00:05.526168,True,False,111


In [4]:
flights.url

'https://www.google.com/travel/flights?q=Flights%20to%20LAX%20from%20MUC%20on%202023-10-28%20oneway&curr=EUR&gl=IT'